In [92]:
#import dependencies 
import pandas as pd 
import requests 
import json 
import snowflake.connector
import config 
import os 
import time 

In [110]:
#cereate connection to specigic database/schema 
con = snowflake.connector.connect(
    user=config.keys['user'],
    password=config.keys['password'],
    account=config.keys['account'],
    warehouse = 'WAREHOUSE_BLUE',
    database  = 'DATABASE_BLUE',
    schema = 'SCHEMA_BLUE'
)

In [4]:
create_table_query = (
    """create or replace TABLE DATABASE_BLUE.SCHEMA_BLUE.BLUE4 (
	"fiscal_year" VARCHAR(50),
	"payroll_number" VARCHAR(50),
	"agency_name" VARCHAR(50),
	"last_name" VARCHAR(50),
	"first_name" VARCHAR(50),
	"mid_init" VARCHAR(50),
	"agency_start_date" VARCHAR(50),
	"work_location_borough" VARCHAR(50),
	"title_description" VARCHAR(50),
	"leave_status_as_of_june_30" VARCHAR(50),
	"base_salary" VARCHAR(50),
	"pay_basis" VARCHAR(50),
	"regular_hours" VARCHAR(50),
	"regular_gross_paid" VARCHAR(50),
	"ot_hours" VARCHAR(50),
	"total_ot_paid" VARCHAR(50),
	"total_other_pay" VARCHAR(50)
);""" 
) 

# execute
con.cursor().execute(create_table_query)

In [66]:
#get dataset info for funciton 
#get row count for dataset 
data_size = requests.get("https://data.cityofnewyork.us/resource/k397-673e.json?$select=count(*)")
data_size.json()
data_size = data_size.json()[0]['count']
data_size

'5662713'

In [ ]:
#get years in dataset
years = requests.get("https://data.cityofnewyork.us/resource/k397-673e.json?$select=distinct fiscal_year")
years = years.json()
years = [i['fiscal_year'] for i in years]
years.sort()

In [67]:
data_size

'5662713'

In [77]:
response.status_code

200

In [83]:
print(3)

3


In [112]:
for i in years: 
    #define function variables 
    url = f"https://data.cityofnewyork.us/resource/k397-673e.json?$limit={300}&$where=fiscal_year={i}"
    #using dataset size as arbitrary limit
    response = requests.get(url)
    
    if response.status_code != 200:
        break 
        
    file_path = f'temp/{i}_data_sample.csv'
    #save data to temp folder as csv    
    dataframe = pd.DataFrame(response.json())
    dataframe.to_csv(file_path, index=False)   
    
    print(f'{i}:rows:{len(dataframe)}')
    
    #stage file in snowflake 
    con.cursor().execute(f'PUT file://{file_path}* @%BLUE4')
    
    #free up memory, clear out variable and delete temp file 
    dataframe = None
    if os.path.isfile(file_path):
        os.remove(file_path)
    
    #
    time.sleep(10)

2014:rows:300
2015:rows:300
2016:rows:300
2017:rows:300
2018:rows:300
2019:rows:300
2020:rows:300
2021:rows:300
2022:rows:300
2023:rows:300


In [107]:
#include "/" at the end of dir path 
def clear_dir(dir_path): 
    for i in os.listdir(dir_path):
        os.remove(f"{dir_path}{i}")
    

In [111]:
clear_dir('temp/')